# Max Flow Applications

The purpose of this assignment is to investigate applications of finding a Max Flow. The problem asks you to design and implement an algorithm for shipping a material between nodes with different supply and demand requirements.

* Please write code *only* in the bodies of the two functions, that is, following the TODO comments.
* Be careful not to use varibles defined outside of the functions.
* Breaking the two above rules may lead to 0 grades.

## Movie distribution

First solve Problem 3 from HW3-theoretical. 

Now suppose a movie distributor would like to ship a copy of a film from CA to every other state. There are therefore 48 units to ship out of CA, and each other state receives 1 unit. 

The dataset contiguous-usa.dat lists the adjacent states in the US. Each line lists two adjacent states; thus AK and HI are omitted, but DC is included in the data. The following code reads in the graph of US states.

In [1]:
import networkx as nx
G = nx.Graph()

usa = open('contiguous-usa.dat')
for line in usa:
    s1, s2 = line.strip().split()
    G.add_edge(s1, s2)

We now encode the demands into the graph.

In [2]:
for state in G.nodes():
    if state != 'CA':
        G.node[state]['demand'] = 1
G.node['CA']['demand'] = -48

We will assign a uniform capacity of 16 to each edge. Since CA has only three adjacent states, this is the smallest possible uniform capacity that allows one to ship all 48 units out of CA. As we have created an undirected graph, and flows have directions, we first convert the graph to a directed graph.

In [3]:
G = nx.DiGraph(G)
uniform_capacity = 16
for (s1, s2) in G.edges():
    G.edge[s1][s2]['capacity'] = uniform_capacity

Complete the following function to implement your algorithm to find a flow with demands. Your function should work correctly for any input, not just the movie instance considered here. As always, you are encouraged to define auxiliary functions as needed for clarity.

In [4]:
def flow_with_demands(graph):
    """Computes a flow with demands over the given graph.
    
    Args:
        graph: A directed graph with nodes annotated with 'demand' properties and edges annotated with 'capacity' 
            properties.
        
    Returns:
        A dict of dicts containing the flow on each edge. For instance, flow[s1][s2] should provide the flow along
        edge (s1, s2).
        
    Raises:
        NetworkXUnfeasible: An error is thrown if there is no flow satisfying the demands.
    """
    def checkIsfeasible(graph):
        net_demand = 0
        net_supply = 0
        for state in graph.nodes():
            if graph.node[state]['demand'] >0:
                net_demand += graph.node[state]['demand']
            else:
                net_supply += graph.node[state]['demand']
        if abs(net_demand) != abs(net_supply):
            raise nx.NetworkXUnfeasible()
        else:
            return abs(net_supply)
            
    #check if supply(+ve demands) = demand constraint is met          
    totalSupply = checkIsfeasible(graph)
    
    #create working copy
    G_new= graph.copy()
    superSource = 'source'
    superSink = 'sink'
    G_new.add_node(superSource)
    G_new.add_node(superSink)
    
    #convert demands to capacitated edges in graph
    for state in graph.nodes():
        #print state
        if state not in [superSink,superSource]:
            if (graph.node[state]['demand']) >0:
                G_new.add_edge(state,superSink)
                G_new.edge[state][superSink]['capacity'] = abs(graph.node[state]['demand'])
            if (graph.node[state]['demand']) <0:
                G_new.add_edge(superSource,state)
                G_new.edge[superSource][state]['capacity'] = abs(graph.node[state]['demand'])
    
    #run maxflow
    max_flow_value = 0
    max_flow_dict = {}
    try:
        max_flow_value,max_flow_dict = nx.maximum_flow(G_new, s=superSource, t=superSink, capacity='capacity')
    except:
        raise nx.NetworkXUnfeasible()
        
    #print max_flow_value
    
    #for demand constraint to be met,these must be equal
    if max_flow_value != totalSupply:
        raise nx.NetworkXUnfeasible()
    
    #sanity check- flow dic is not empty
    if len(max_flow_dict)<=0:
        raise nx.NetworkXUnfeasible()
    
    #format the flow dict to remove the added edges and nodes
    del max_flow_dict[superSource]
    del max_flow_dict[superSink]
    
    for startVertex in max_flow_dict:
        endVertices = max_flow_dict[startVertex]
        if superSink in endVertices.keys():
            del endVertices[superSink]
            
    #print max_flow_dict    
    return max_flow_dict


To verify that your solution is correct, implement a function that computes the total flow into each node (which will be negative for supply nodes).

In [5]:
def divergence(flow):
    """Computes the total flow into each node according to the given flow dict.
    
    Args:
        flow: the flow dict recording flow between nodes.
        
    Returns:
        A dict of the net flow into each node.
    """
    netInFlow ={}
    ##in the flows,we have nested dictionary.
    #the outer key is the start vertex for an outgoing edge thus outgoing flow for the node
    #the inner keys are the nodes recieving flows
    for startVertex in flow:
        if startVertex in netInFlow.keys():
            netInFlow[startVertex] -= sum(flow[startVertex].values())
        else:
            netInFlow[startVertex] = -1 * sum(flow[startVertex].values())
        for endvertex in flow[startVertex]:
            if endvertex in netInFlow.keys():
                netInFlow[endvertex] += flow[startVertex][endvertex]
            else:
                netInFlow[endvertex] = flow[startVertex][endvertex]
    return netInFlow



The following code performs a sanity check on your function (but does not completely confirm correctness).

In [6]:
flow = flow_with_demands(G)
div = divergence(flow)

print "Flow satisfies all demands:", all(div[n] == G.node[n]['demand'] for n in G.nodes())

Flow satisfies all demands: True


In [7]:
#div

In [8]:
#flow